# Transition counts of segments
Count the transitions between different diffusion states of segments within trajectories.

In [1]:
from pySPT.widgets import widgetTransitionCounts
from pySPT.widgets import loadTransitionCounts
from pySPT.Analysis import transitionCount
import tqdm.notebook as tqdm
import pandas as pd
import os

## Load files from directories

Define the path to the directory containing h5 files from trackAnalysis/trackStatistics and the corresponding .tracked.csv files from swift. Each target has two files (a h5 and a .tracked.csv) and the file naming must be the same for each target in order to match the files.

In [2]:
input_paths = [r""]  # adjust the default parameters
n_dirs = 1
dir_objects = []
for n_dir in range(n_dirs):
    file_dir = widgetTransitionCounts.DefineDirectory("Directory", value=input_paths[n_dir])
    file_dir.dir_button.on_click(file_dir.open_dir)
    file_dir.dir_box.observe(file_dir.change_dir_box)
    display(file_dir.dir_box, file_dir.dir_button)
    dir_objects.append(file_dir)

Text(value='', description='Directory', placeholder='directory to be searched in', style=DescriptionStyle(desc…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

## Parameters

*Diffusion states*: Number of diffusion type to distinguish: 3 = immobile+notype = one immobile population, confined, free; 4 = immobile, confined, free, notype <br/>
*Mask value*: In trackAnalysis/trackStatistics only segments above a set length are classified into diffusion states. Mask unclassified short segments with a length in frames <= mask value for counting. A mask value = 0 means no masking is applied. <br/>
*Save dir*: Specify the directory in which the results will be saved.<br/>
*Save folder*: Specify the name of the results folder.<br/>
When re-running the analysis, specify a new *Save dir* or *Save folder* as data will not be overwritten.

Already run the analysis and directly want to visualize the results? Define the paths to the transition_counts.txt and trajectory_table.txt files under *Counts file* and *Trajectory file*. <br/>

In [3]:
save_dir = r""  # adjust the default parameters
save_folder = r"transition_counting"
counts_file = r""
trajectory_file = r""
widgets_params = widgetTransitionCounts.Parameter(3, 0, counts_file, trajectory_file, save_dir, save_folder)
display(widgets_params.n_diff_states_box, widgets_params.mask_box, widgets_params.save_dir_box, widgets_params.save_folder_box)
print("*"*38)
display(widgets_params.counts_file_box, widgets_params.trajectory_file_box)

Text(value='3', description='Diffusion states', placeholder='number of diffusion states', style=DescriptionSty…

Text(value='0', description='Mask value', placeholder='directory to save results', style=DescriptionStyle(desc…

Text(value='', description='Save dir', placeholder='directory to save results', style=DescriptionStyle(descrip…

Text(value='transition_counting', description='Save folder', placeholder='name of results folder', style=Descr…

**************************************


Text(value='', description='Counts file', placeholder='path to counts file', style=DescriptionStyle(descriptio…

Text(value='', description='Trajectory file', placeholder='path to trajectory table file', style=DescriptionSt…

## Run Analysis

Run, plot and save the analysis.

In [4]:
stats = transitionCount.Statistic()
widgets_run = widgetTransitionCounts.RunAnalysis()
display(widgets_run.run_analysis_button)
def run_analysis(event):
    widgets_run.create_clear_output()
    display(widgets_run.run_analysis_button)
    dir_paths = [dir_object.dir_box.value for dir_object in dir_objects if dir_object.dir_box.value]
    data = loadTransitionCounts.DataLoader(dir_paths)

    if not widgets_params.counts_file_box.value or not widgets_params.trajectory_file_box.value:
        if data.got_files:
            # get counts
            transition_counts = []
            for idx in tqdm.tqdm(range(len(data.h5_files)), desc="Files"):
                transition_count = transitionCount.TransitionCounts(data.h5_names[idx], data.h5_files[idx], data.tracked_files[idx], int(widgets_params.n_diff_states_box.value), int(widgets_params.mask_box.value))
                transition_counts.append(transition_count)
            # transfer to stats class
            stats.counts = [i.counts for i in transition_counts]
            stats.trajectory_tables = [i.trajectory_table for i in transition_counts]
            stats.sizes = [i.cell_size for i in transition_counts]
            stats.n_diffusion_types = int(widgets_params.n_diff_states_box.value)
            # save as txt file
            os.mkdir(widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value)
            save_names = [i for i in data.h5_names]
            transitionCount.save_counts(transition_counts, save_names, widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value)
            transitionCount.save_trajectory_tables([i.trajectory_table for i in transition_counts], widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value)
            transitionCount.save_mask_value(widgets_params.mask_box.value, widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value)
        else:
            print("Directory does not contain required files.")
    else:
        # read counts from txt file
        pd_file = pd.read_csv(widgets_params.counts_file_box.value, sep="\t")
        stats.counts = [list(pd_file.iloc[i,2:]) for i in range(len(pd_file))]
        stats.sizes = list(pd_file.iloc[:,1])
        stats.n_diffusion_types = 4 if pd_file.shape[1] == 19 else 3
        trajectory_table = pd.read_csv(widgets_params.trajectory_file_box.value, sep="\t")
        for name in trajectory_table["target.name"].unique():
            stats.trajectory_tables.append(trajectory_table[trajectory_table["target.name"] == name])        
        print("Transition counts files loaded.")

    stats.save_dir = widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value
    
    # adjust axis min max limits of plots here, e.g. ylim=[0,1]
    stats.vis_counts(counts="absolute", norm="absolute", ylim=[None, None])
    stats.vis_counts(counts="absolute", norm="global", ylim=[None, None])
    stats.vis_counts(counts="absolute", norm="split", ylim=[None, None])    
    stats.segment_lengths_plot(xlim=[0,None], ylim=[0, None])
    stats.segments_per_trajectory_plot(xlim=[0,10], ylim=[0, None])
    stats.transitions_wo_none_plot(ylim=[0,1])

    print("Analysis executed. Files successfully saved at", widgets_params.save_dir_box.value + "\\" + widgets_params.save_folder_box.value)
        
widgets_run.run_analysis_button.on_click(run_analysis)

Button(description='run & save', style=ButtonStyle(), tooltip='run analysis')